In [62]:
import numpy as np
import pandas as pd
import os
import sys

from IPython.display import display
from scipy.cluster import hierarchy as hac

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [63]:
''' LOAD INFORMATION'''
input_csv = '../../data/fermi_4FGL_associations_ext_GRPHorder+data.csv'
data_in = pd.read_csv(input_csv)
data_in.drop(columns='Unnamed: 0', inplace=True)
DROP_COLUMNS = ['flux', 'flux_err', 'flux_E', 'flux_E_err', 'redshift_flag', 'median_optical_flux', 'num_optical_flux', 'PL_index', 
                'PL_index_err', 'variability_index', 'frac_variability', 'frac_variability_err',]
data_in.drop(columns=DROP_COLUMNS, inplace=True)

obj_analyze = np.loadtxt('objects_to_analyze_reduced.dat').astype(np.int)
num_analyze = len(obj_analyze)
print('Total number of objects: '+str(num_analyze))

# cluster_file = 'hac_corr_cc+acg+aco_L2weighted_10clusters_v2.dat'
cluster_file = 'hac_corr_cc_complete_10clusters_v10.dat'
clusters = np.loadtxt(cluster_file).astype(np.int)
num_clusters = clusters.max()+1
print('Total number of clusters: '+str(num_clusters))

in_cluster = np.array([200, 143, 63, 12, 52, 8, 677, 622, 458, 23])
# in_cluster = np.array([59,8,45,116,13,52,174,190,272,23,39,12,290,113,90,428,160,69,0,368,63])
cluster_order = np.zeros(10).astype(np.int)
for ii in range(0,10):
	cluster_order[ii] = clusters[obj_analyze == in_cluster[ii]]
# cluster_order = np.linspace(0,19, num=20).astype(np.int)
print(cluster_order)

Total number of objects: 549
Total number of clusters: 10
[ 1  1 -1 -1  5  5 -1  6  0  0 -1  2  0 -1  8  8 -1  0 -1  3]


In [51]:
re_ordered_cluster = np.ones(len(clusters)).astype(np.int)*-1
# print(clusters)
for ii in range(0,num_clusters):
	re_ordered_cluster[clusters == cluster_order[ii]] = ii 

# print(re_ordered_cluster)
reorder_cluster_file = 'hac_corr_cc_complete_10clusters_v10_reorder.dat'
np.savetxt(reorder_cluster_file, re_ordered_cluster, fmt='%d')

print(re_ordered_cluster[re_ordered_cluster>0])

# display(data_in.head(5))
# display(data_in.columns.tolist())

[ 9  7 16 19 19  5  4 18  6  1 17  9  3  3 12  2 12 14  8 18 11 11  4 17
  1 15  1  7 15 13 10  2 10  6 14  8 13 16  5  1]


In [52]:
''' COMBINE ACs + CCs '''

autocorr_opt_err_dir = '../../2001/output/auto_opt/'
autocorr_opt_sig_dir = '../../2001/output/autocorr_opt+opt/'
opt_file_num = 0

autocorr_gam_err_dir = '../../2001/output/auto_gam/'
autocorr_gam_sig_dir = '../../2001/output/autocorr_gam+gam/'
gam_file_num = 0

cc_dir = '../../2001/output/cross_corr/'
cross_object_cc_dir = '../../2001/output/cross_object_gam+opt/'
cc_file_num = 0

out_dat = './dat/'
obj_num_out = 9100 + np.arange(0,num_clusters).astype(np.int)
out_num_out = 0
# print(obj_num_out)

In [53]:
'''

FUNCTIONS

'''

def read_all_err(filenames):
	num = len(filenames)
	
	for ii in range(0,num):
		read_in = np.loadtxt(filenames[ii])

		time = read_in[0]
		data_err = np.ma.masked_equal(read_in[1:], -1.1)

		cor_mean = data_err[0]
		cor_err_d = cor_mean-data_err[1]+1.e-10
		cor_err_u = data_err[2]-cor_mean+1.e-10
		
		if ii == 0:
			time_tot = time
			cor_mean_tot = np.array([cor_mean])
			cor_err_d_tot = np.array([cor_err_d])
			cor_err_u_tot = np.array([cor_err_u])
		else:
			cor_mean_tot = np.append(cor_mean_tot, np.array([cor_mean]), axis=0)
			cor_err_d_tot = np.append(cor_err_d_tot, np.array([cor_err_d]), axis=0)
			cor_err_u_tot = np.append(cor_err_u_tot, np.array([cor_err_u]), axis=0)
	
	return [time_tot, cor_mean_tot, cor_err_d_tot, cor_err_u_tot]

def combine_all_err(all_arr):
	time = all_arr[0]
	mean_all = all_arr[1]
	err_d_all = all_arr[2]
	err_u_all = all_arr[3]
	
	err_avg = (err_d_all+err_u_all)/2
	mean = np.sum(mean_all/err_avg**2, axis=0)/np.sum(1/err_avg**2, axis=0) 
	err_d = 1/np.sqrt(np.sum(1/err_d_all**2, axis=0))
	err_u = 1/np.sqrt(np.sum(1/err_u_all**2, axis=0))
	
	return np.array([time, mean, err_d, err_u])

def read_all_sig(filenames):
	num = len(filenames)
	initialized_arr = 0
	for ii in range(0,num):
		try:
			read_in = np.loadtxt(filenames[ii])
			time = read_in[0]
			data_sig = np.ma.masked_equal(read_in[1:], -1.1)
		except:
			continue
			
		sig_mean = data_sig[0]
		sig_sig1_d = data_sig[1]
		sig_sig1_u = data_sig[2]
		sig_sig2_d = data_sig[3]
		sig_sig2_u = data_sig[4]
		sig_sig3_d = data_sig[5]
		sig_sig3_u = data_sig[6]
		sig_siga_d = data_sig[7]
		sig_siga_u = data_sig[8]
		
		if initialized_arr == 0:
			time_tot = time
			sig_mean_tot = np.array([sig_mean])
			sig_sig1_d_tot = np.array([sig_sig1_d])
			sig_sig1_u_tot = np.array([sig_sig1_u])
			sig_sig2_d_tot = np.array([sig_sig2_d])
			sig_sig2_u_tot = np.array([sig_sig2_u])
			sig_sig3_d_tot = np.array([sig_sig3_d])
			sig_sig3_u_tot = np.array([sig_sig3_u])
			sig_siga_d_tot = np.array([sig_siga_d])
			sig_siga_u_tot = np.array([sig_siga_u])
			
			initialized_arr+= 1
		else:
			sig_mean_tot = np.append(sig_mean_tot, np.array([sig_mean]), axis=0)
			sig_sig1_d_tot = np.append(sig_sig1_d_tot, np.array([sig_sig1_d]), axis=0)
			sig_sig1_u_tot = np.append(sig_sig1_u_tot, np.array([sig_sig1_u]), axis=0)
			sig_sig2_d_tot = np.append(sig_sig2_d_tot, np.array([sig_sig2_d]), axis=0)
			sig_sig2_u_tot = np.append(sig_sig2_u_tot, np.array([sig_sig2_u]), axis=0)
			sig_sig3_d_tot = np.append(sig_sig3_d_tot, np.array([sig_sig3_d]), axis=0)
			sig_sig3_u_tot = np.append(sig_sig3_u_tot, np.array([sig_sig3_u]), axis=0)
			sig_siga_d_tot = np.append(sig_siga_d_tot, np.array([sig_siga_d]), axis=0)
			sig_siga_u_tot = np.append(sig_siga_u_tot, np.array([sig_siga_u]), axis=0)
	
	output = [time, sig_mean_tot, 
			  sig_sig1_d_tot, sig_sig1_u_tot, 
			  sig_sig2_d_tot, sig_sig2_u_tot, 
			  sig_sig3_d_tot, sig_sig3_u_tot, 
			  sig_siga_d_tot, sig_siga_u_tot]
	
	return output

def combine_all_sig(all_arr):
	time = all_arr[0]
	sig_mean_tot = all_arr[1]
	sig_sig1_d_tot = all_arr[2]
	sig_sig1_u_tot = all_arr[3]
	sig_sig2_d_tot = all_arr[4]
	sig_sig2_u_tot = all_arr[5]
	sig_sig3_d_tot = all_arr[6]
	sig_sig3_u_tot = all_arr[7]
	sig_siga_d_tot = all_arr[8]
	sig_siga_u_tot = all_arr[9]
	
	sig_mean = np.average(sig_mean_tot, axis=0)
	sig_sig1_d = -1/np.sqrt(np.sum(1/sig_sig1_d_tot**2, axis=0))
	sig_sig1_u = 1/np.sqrt(np.sum(1/sig_sig1_u_tot**2, axis=0))
	sig_sig2_d = -1/np.sqrt(np.sum(1/sig_sig2_d_tot**2, axis=0))
	sig_sig2_u = 1/np.sqrt(np.sum(1/sig_sig2_u_tot**2, axis=0))
	sig_sig3_d = -1/np.sqrt(np.sum(1/sig_sig3_d_tot**2, axis=0))
	sig_sig3_u = 1/np.sqrt(np.sum(1/sig_sig3_u_tot**2, axis=0))
	sig_siga_d = -1/np.sqrt(np.sum(1/sig_siga_d_tot**2, axis=0))
	sig_siga_u = 1/np.sqrt(np.sum(1/sig_siga_u_tot**2, axis=0))
	
	output = np.array([time, sig_mean,
					  sig_sig1_d, sig_sig1_u,
					  sig_sig2_d, sig_sig2_u,
					  sig_sig3_d, sig_sig3_u,
					  sig_siga_d, sig_siga_u])
	
	return output



In [54]:
''' READ ALL ERR FILES'''
opt_err_all = []
gam_err_all = []
cc_err_all  = []

for ii in range(0,num_clusters):
	obj_in_cluster = obj_analyze[np.argwhere(clusters == ii)].T[0]
	num_obj_in_cluster = len(obj_in_cluster)
	# print(obj_in_cluster)
	
	opt_err_filenames = []
	gam_err_filenames = []
	cc_err_filenames  = []
	for jj in range(0,num_obj_in_cluster):
		opt_err_filenames+= [autocorr_opt_err_dir+'object'+str(obj_in_cluster[jj]).zfill(4)+'_auto_opt_stats'+str(opt_file_num).zfill(3)+'.dat']
		gam_err_filenames+= [autocorr_gam_err_dir+'object'+str(obj_in_cluster[jj]).zfill(4)+'_auto_gam_stats'+str(gam_file_num).zfill(3)+'.dat']
		cc_err_filenames += [cc_dir+'object'+str(obj_in_cluster[jj]).zfill(4)+'_stats'+str(cc_file_num).zfill(3)+'.dat']
	
	opt_err_all += [read_all_err(opt_err_filenames)]
	gam_err_all += [read_all_err(gam_err_filenames)]
	cc_err_all  += [read_all_err(cc_err_filenames)]


''' COMBINE ALL THE DATA '''
cluster_opt = []
cluster_gam = []
cluster_cc  = []

for ii in range(0,num_clusters):
	cluster_opt += [combine_all_err(opt_err_all[ii])]
	cluster_gam += [combine_all_err(gam_err_all[ii])]
	cluster_cc  += [combine_all_err(cc_err_all[ii])]
	
	
''' SAVE CLUSTERS AS INDIVIDUAL OBJECTS '''
for ii in range(0,num_clusters):
	out_opt_name = out_dat+'object'+str(obj_num_out[ii]).zfill(4)+'_auto_opt_stats'+str(out_num_out).zfill(3)+'.dat'
	out_gam_name = out_dat+'object'+str(obj_num_out[ii]).zfill(4)+'_auto_gam_stats'+str(out_num_out).zfill(3)+'.dat'
	out_cc_name  = out_dat+'object'+str(obj_num_out[ii]).zfill(4)+'_stats'+str(out_num_out).zfill(3)+'.dat'
	
	np.savetxt(out_opt_name, cluster_opt[ii])
	np.savetxt(out_gam_name, cluster_gam[ii])
	np.savetxt(out_cc_name , cluster_cc[ii])

In [55]:
''' ANALYZE ALL SIG '''
opt_sig_all = []
gam_sig_all = []
cc_sig_all  = []

for ii in range(0,num_clusters):
	opt_sig_filenames = []
	gam_sig_filenames = []
	cc_sig_filenames  = []
	for jj in range(0,num_obj_in_cluster):
		opt_sig_filenames+= [autocorr_opt_sig_dir+'cross_stats_opt+opt_object'+str(obj_in_cluster[jj]).zfill(4)+'_'+str(opt_file_num).zfill(3)+'.dat']
		gam_sig_filenames+= [autocorr_gam_sig_dir+'cross_stats_gam+gam_object'+str(obj_in_cluster[jj]).zfill(4)+'_'+str(gam_file_num).zfill(3)+'.dat']
		cc_sig_filenames += [cross_object_cc_dir+'cross_object'+str(obj_in_cluster[jj]).zfill(4)+'_all_stats_'+str(cc_file_num).zfill(3)+'.dat']
	
	opt_sig_all+= [opt_sig_filenames]
	gam_sig_all+= [gam_sig_filenames]
	cc_sig_all += [ cc_sig_filenames]
	
cluster_opt_sig = []
cluster_gam_sig = []
cluster_cc_sig  = []

for ii in range(0,num_clusters):
	cluster_opt_sig += [combine_all_sig(read_all_sig(opt_sig_all[ii]))]
	cluster_gam_sig += [combine_all_sig(read_all_sig(gam_sig_all[ii]))]
	cluster_cc_sig  += [combine_all_sig(read_all_sig( cc_sig_all[ii]))]


''' SAVE CLUSTERS AS INDIVIDUAL OBJECTS '''
for ii in range(0,num_clusters):
	out_opt_name = out_dat+'cross_stats_opt+opt_object'+str(obj_num_out[ii]).zfill(4)+'_'+str(out_num_out).zfill(3)+'.dat'
	out_gam_name = out_dat+'cross_stats_gam+gam_object'+str(obj_num_out[ii]).zfill(4)+'_'+str(out_num_out).zfill(3)+'.dat'
	out_cc_name  = out_dat+'cross_object'+str(obj_num_out[ii]).zfill(4)+'_all_stats_'+str(out_num_out).zfill(3)+'.dat'
	
	np.savetxt(out_opt_name, cluster_opt_sig[ii])
	np.savetxt(out_gam_name, cluster_gam_sig[ii])
	np.savetxt(out_cc_name , cluster_cc_sig[ii])
	

C:\Users\psyko\Anaconda3\lib\site-packages\ipykernel_launcher.py:111: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\psyko\Anaconda3\lib\site-packages\ipykernel_launcher.py:112: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\psyko\Anaconda3\lib\site-packages\ipykernel_launcher.py:113: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\psyko\Anaconda3\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\psyko\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\psyko\Anaconda3\lib\site-packages\ipykernel_launcher.py:116: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\psyko\Anaconda3\lib\site-packages\ipykernel_launcher.py:117: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\psyko\Anaconda3\lib\site-packages\ipykernel_launcher.py:118: RuntimeWarning: divide by zero enc

In [58]:
LABELS = ['A','B','C','D','E','F','G','H','I','J']
LABELS = LABELS[::-1]
LABELS = np.array(LABELS)

# LABELS = ['A','B','C','D','E','F','G','H','I','J','A','B','C','D','E','F','G','H','I','J']
# LABELS = LABELS[::-1]
# LABELS = np.array(LABELS)

In [60]:
'''

PLOT ALL CLUSTERS ON ONE PLOT

'''

###
### PLOT
###
font = {'family' : 'serif',
		'weight' : 'normal',
		'size'   : 5.5}

matplotlib.rc('font', **font)
matplotlib.rcParams['mathtext.fontset']='dejavuserif'

cmap2 = plt.get_cmap('gist_stern')
cmap  = plt.get_cmap('brg')#'gnuplot2')#'BuPu_r')
fig = plt.figure()

gs = gridspec.GridSpec(4, 1)
gs.update(wspace=0.0, hspace=0.0)
plt.subplots_adjust(hspace=0.001)

ax0 = plt.subplot(gs[0,0])
ax1 = plt.subplot(gs[1,0])
ax2 = plt.subplot(gs[2,0])
ax3 = plt.subplot(gs[3,0])

for axis in ['top','bottom','left','right']:
	ax0.spines[axis].set_linewidth(1)
ax0.tick_params(which='major',width=0.5, length=8, labelsize='small', direction='in')
ax0.tick_params(which='minor',width=0.25, length=5, direction='in')

for axis in ['top','bottom','left','right']:
	ax1.spines[axis].set_linewidth(1)
ax1.tick_params(which='major',width=0.5, length=8, labelsize='small', direction='in')
ax1.tick_params(which='minor',width=0.25, length=5, direction='in')

for axis in ['top','bottom','left','right']:
	ax2.spines[axis].set_linewidth(1)
ax2.tick_params(which='major',width=0.5, length=8, labelsize='small', direction='in')
ax2.tick_params(which='minor',width=0.25, length=5, direction='in')

for axis in ['top','bottom','left','right']:
	ax3.spines[axis].set_linewidth(1)
ax3.tick_params(which='major',width=0.5, length=8, labelsize='small', direction='in')
ax3.tick_params(which='minor',width=0.25, length=5, direction='in')

MSIZE = 0.6
ALPHA = 0.2
ELINE = 0.5

ALPHA_FILL_DAT = 0.3
alpha_fill = 0.2
lwidth_fill = 0.1

cmap_col = cmap(np.linspace(0,1,num=10))

for ii in range(0,num_clusters):
	COLOR = cmap_col[cluster_order == ii]
	
	## optical AC
	data = cluster_opt[ii]
	time = data[0]
	cor_mean = data[1]
	cor_err_d = data[2]
	cor_err_u = data[3]
	ax0.fill_between(-time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE)
	ax2.fill_between(-time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE)
	
	## gamma AC
	data = cluster_gam[ii]
	time = data[0]
	cor_mean = data[1]
	cor_err_d = data[2]
	cor_err_u = data[3]
	ax0.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE)
	ax2.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE)
	
	## CC
	data = cluster_cc[ii]
	time = data[0]
	cor_mean = data[1]
	cor_err_d = data[2]
	cor_err_u = data[3]
	ax1.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE)
	ax3.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE)
	
	

ax0.set_xlim([-1299.999,2999.999])
ax0.set_ylim([-0.399,0.999])

ax1.set_xlim([-1299.999,2999.999])
ax1.set_ylim([-0.399,0.499])

ax2.set_xlim([-49.99,49.99])
ax2.set_ylim([0.001,0.999])

ax3.set_xlim([-49.99,49.99])
ax3.set_ylim([0.001,0.499])

ax0.text(-1000., 0.85, r'AC', verticalalignment='center', horizontalalignment='left', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))		
ax0.text(150., 0.85, r'$\gamma$-ray$\rightarrow$', verticalalignment='center', horizontalalignment='left', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))
ax0.text(-150., 0.85, r'$\leftarrow$Optical', verticalalignment='center', horizontalalignment='right', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))

ax1.text(-1000., 0.4, r'CC', verticalalignment='center', horizontalalignment='left', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))		
ax1.text(150., 0.4, r'$\gamma$-ray lead$\rightarrow$', verticalalignment='center', horizontalalignment='left', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))
ax1.text(-150., 0.4, r'$\leftarrow$Optical lead', verticalalignment='center', horizontalalignment='right', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))

ax0.set_ylabel(r'$r_s$')
ax1.set_ylabel(r'$r_s$')
ax2.set_ylabel(r'$r_s$')
ax2.set_xlabel(r'Days [$\gamma$-ray leading]')

# ax1.yaxis.set_ticks_position('right')
plt.setp(ax1.get_xticklabels(), ha='right', rotation=45)
ax1.zorder=1000

plt.tight_layout()
plot_name = 'plot_combine_corr_'
for n in range(0,100):
	if os.path.isfile(plot_name+str(n).zfill(2)+'.png'):
		continue
	else:
		plt.savefig(plot_name+str(n).zfill(2)+'.png', bbox_inches='tight', dpi=400)
		break
plt.close()


In [61]:
'''PLOT INDIVIDUAL CLUSTERS'''

###
### PLOT
###
font = {'family' : 'serif',
		'weight' : 'normal',
		'size'   : 10}

matplotlib.rc('font', **font)
matplotlib.rcParams['mathtext.fontset']='dejavuserif'

cmap2 = plt.get_cmap('gist_stern')
cmap  = plt.get_cmap('brg')#'gnuplot2')#'BuPu_r')


for ii in range(0,num_clusters):

	fig = plt.figure()

	gs = gridspec.GridSpec(2, 1)
	gs.update(wspace=0.0, hspace=0.0)
	plt.subplots_adjust(hspace=0.001)

	ax0 = plt.subplot(gs[0,0])
	ax1 = plt.subplot(gs[1,0])
# 	ax2 = plt.subplot(gs[2,0])
# 	ax3 = plt.subplot(gs[3,0])

	for axis in ['top','bottom','left','right']:
		ax0.spines[axis].set_linewidth(1)
	ax0.tick_params(which='major',width=0.5, length=8, labelsize='small', direction='in')
	ax0.tick_params(which='minor',width=0.25, length=5, direction='in')

	for axis in ['top','bottom','left','right']:
		ax1.spines[axis].set_linewidth(1)
	ax1.tick_params(which='major',width=0.5, length=8, labelsize='small', direction='in')
	ax1.tick_params(which='minor',width=0.25, length=5, direction='in')

# 	for axis in ['top','bottom','left','right']:
# 		ax2.spines[axis].set_linewidth(1)
# 	ax2.tick_params(which='major',width=0.5, length=8, labelsize='small', direction='in')
# 	ax2.tick_params(which='minor',width=0.25, length=5, direction='in')

# 	for axis in ['top','bottom','left','right']:
# 		ax3.spines[axis].set_linewidth(1)
# 	ax3.tick_params(which='major',width=0.5, length=8, labelsize='small', direction='in')
# 	ax3.tick_params(which='minor',width=0.25, length=5, direction='in')

	MSIZE = 0.6
	ALPHA = 0.2
	ELINE = 0.5

	ALPHA_FILL_DAT = 0.7
	alpha_fill = 0.2
	lwidth_fill = 0.1
	LWIDTH = 0.075
	ALPHA_BK = 0.05

	COLOR = cmap_col[cluster_order == ii]
	LABEL = LABELS[cluster_order == ii]
	LABEL = LABEL[0]
	
	''' 
	optical AC 
	'''
	data = cluster_opt[ii]
	time = data[0]
	cor_mean = data[1]
	cor_err_d = data[2]
	cor_err_u = data[3]
	
	data_sig = cluster_opt_sig[ii]
	sig_mean = data_sig[1]
	sig_sig1_d = data_sig[2]
	sig_sig1_u = data_sig[3]
	sig_sig2_d = data_sig[4]
	sig_sig2_u = data_sig[5]
	sig_sig3_d = data_sig[6]
	sig_sig3_u = data_sig[7]
	sig_siga_d = data_sig[8]
	sig_siga_u = data_sig[9]
	
# 	sigma3_corr = cor_mean > sig_sig3_u
# 	sigma3_anti = cor_mean < sig_sig3_d

# 	sigma2_corr = cor_mean > sig_sig2_u
# 	sigma2_anti = cor_mean < sig_sig2_d
# 	sigma2_corr = np.logical_xor(sigma2_corr, sigma3_corr)
# 	sigma2_anti = np.logical_xor(sigma2_anti, sigma3_anti)

# 	time_sig3_corr = time[sigma3_corr]
# 	time_sig3_anti = time[sigma3_anti]
# 	time_sig2_corr = time[sigma2_corr]
# 	time_sig2_anti = time[sigma2_anti]
	
	ax0.fill_between(-time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE, label='Cluster '+LABEL)
# 	ax2.fill_between(-time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE)
	
	ax0.fill_between(-time, sig_sig1_d, sig_sig1_u, facecolor=cmap2(0.05), edgecolor=cmap2(0.05), alpha=alpha_fill, linewidth=lwidth_fill, zorder=0.914)
	ax0.fill_between(-time, sig_sig2_d, sig_sig2_u, facecolor=cmap2(0.15), edgecolor=cmap2(0.15), alpha=alpha_fill, linewidth=lwidth_fill, zorder=0.913)
	ax0.fill_between(-time, sig_sig3_d, sig_sig3_u, facecolor=cmap2(0.25), edgecolor=cmap2(0.25), alpha=alpha_fill, linewidth=lwidth_fill, zorder=0.912)
	
# 	for tc in time_sig3_corr:
# 		ax0.axvline(x=-tc, color=cmap2(0.02), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.9)
# 	for tc in time_sig3_anti:
# 		ax0.axvline(x=-tc, color=cmap2(0.02), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.9)

# 	for tc in time_sig2_corr:
# 		ax0.axvline(x=-tc, color=cmap2(0.60), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.8)
# 	for tc in time_sig2_anti:
# 		ax0.axvline(x=-tc, color=cmap2(0.60), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.8)
	
	data_cluster = opt_err_all[ii]
	time = data_cluster[0]
	for jj in range(0,len(data_cluster[1])):
		cor_mean = data_cluster[1][jj]
		cor_err_d = data_cluster[2][jj]
		cor_err_u = data_cluster[3][jj]
		
		ax0.fill_between(-time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_BK, facecolor='grey', edgecolor='grey', linewidth=ELINE, zorder=0.9)
# 		ax2.fill_between(-time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_BK, facecolor='grey', edgecolor='grey', linewidth=ELINE, zorder=0.9)
		
	
	''' 
	gamma AC
	'''
	data = cluster_gam[ii]
	time = data[0]
	cor_mean = data[1]
	cor_err_d = data[2]
	cor_err_u = data[3]
	
	data_sig = cluster_gam_sig[ii]
	sig_mean = data_sig[1]
	sig_sig1_d = data_sig[2]
	sig_sig1_u = data_sig[3]
	sig_sig2_d = data_sig[4]
	sig_sig2_u = data_sig[5]
	sig_sig3_d = data_sig[6]
	sig_sig3_u = data_sig[7]
	sig_siga_d = data_sig[8]
	sig_siga_u = data_sig[9]
	
# 	sigma3_corr = cor_mean > sig_sig3_u
# 	sigma3_anti = cor_mean < sig_sig3_d

# 	sigma2_corr = cor_mean > sig_sig2_u
# 	sigma2_anti = cor_mean < sig_sig2_d
# 	sigma2_corr = np.logical_xor(sigma2_corr, sigma3_corr)
# 	sigma2_anti = np.logical_xor(sigma2_anti, sigma3_anti)

# 	time_sig3_corr = time[sigma3_corr]
# 	time_sig3_anti = time[sigma3_anti]
# 	time_sig2_corr = time[sigma2_corr]
# 	time_sig2_anti = time[sigma2_anti]
	
	
	ax0.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE)
# 	ax2.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE)
	
	ax0.fill_between(time, sig_sig1_d, sig_sig1_u, facecolor=cmap2(0.05), edgecolor=cmap2(0.05), alpha=alpha_fill, linewidth=lwidth_fill, zorder=0.914)
	ax0.fill_between(time, sig_sig2_d, sig_sig2_u, facecolor=cmap2(0.15), edgecolor=cmap2(0.15), alpha=alpha_fill, linewidth=lwidth_fill, zorder=0.913)
	ax0.fill_between(time, sig_sig3_d, sig_sig3_u, facecolor=cmap2(0.25), edgecolor=cmap2(0.25), alpha=alpha_fill, linewidth=lwidth_fill, zorder=0.912)
	
# 	for tc in time_sig3_corr:
# 		ax0.axvline(x=tc, color=cmap2(0.02), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.9)
# 	for tc in time_sig3_anti:
# 		ax0.axvline(x=tc, color=cmap2(0.02), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.9)

# 	for tc in time_sig2_corr:
# 		ax0.axvline(x=tc, color=cmap2(0.60), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.8)
# 	for tc in time_sig2_anti:
# 		ax0.axvline(x=tc, color=cmap2(0.60), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.8)
	
	data_cluster = gam_err_all[ii]
	time = data_cluster[0]
	for jj in range(0,len(data_cluster[1])):
		cor_mean = data_cluster[1][jj]
		cor_err_d = data_cluster[2][jj]
		cor_err_u = data_cluster[3][jj]
		
		ax0.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_BK, facecolor='grey', edgecolor='grey', linewidth=ELINE, zorder=0.9)
# 		ax2.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_BK, facecolor='grey', edgecolor='grey', linewidth=ELINE, zorder=0.9)
	
	'''
	CC
	'''
	data = cluster_cc[ii]
	time = data[0]
	cor_mean = data[1]
	cor_err_d = data[2]
	cor_err_u = data[3]
	
	data_sig = cluster_cc_sig[ii]
	sig_mean = data_sig[1]
	sig_sig1_d = data_sig[2]
	sig_sig1_u = data_sig[3]
	sig_sig2_d = data_sig[4]
	sig_sig2_u = data_sig[5]
	sig_sig3_d = data_sig[6]
	sig_sig3_u = data_sig[7]
	sig_siga_d = data_sig[8]
	sig_siga_u = data_sig[9]
	
# 	sigma3_corr = cor_mean > sig_sig3_u
# 	sigma3_anti = cor_mean < sig_sig3_d

# 	sigma2_corr = cor_mean > sig_sig2_u
# 	sigma2_anti = cor_mean < sig_sig2_d
# 	sigma2_corr = np.logical_xor(sigma2_corr, sigma3_corr)
# 	sigma2_anti = np.logical_xor(sigma2_anti, sigma3_anti)

# 	time_sig3_corr = time[sigma3_corr]
# 	time_sig3_anti = time[sigma3_anti]
# 	time_sig2_corr = time[sigma2_corr]
# 	time_sig2_anti = time[sigma2_anti]
	
	ax1.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE)
# 	ax3.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_FILL_DAT, facecolor=COLOR, edgecolor=COLOR, linewidth=ELINE)
	
	ax1.fill_between(time, sig_sig1_d, sig_sig1_u, facecolor=cmap2(0.05), edgecolor=cmap2(0.05), alpha=alpha_fill, linewidth=lwidth_fill, zorder=0.914)
	ax1.fill_between(time, sig_sig2_d, sig_sig2_u, facecolor=cmap2(0.15), edgecolor=cmap2(0.15), alpha=alpha_fill, linewidth=lwidth_fill, zorder=0.913)
	ax1.fill_between(time, sig_sig3_d, sig_sig3_u, facecolor=cmap2(0.25), edgecolor=cmap2(0.25), alpha=alpha_fill, linewidth=lwidth_fill, zorder=0.912)
	
# 	for tc in time_sig3_corr:
# 		ax1.axvline(x=tc, color=cmap2(0.02), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.9)
# 	for tc in time_sig3_anti:
# 		ax1.axvline(x=tc, color=cmap2(0.02), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.9)

# 	for tc in time_sig2_corr:
# 		ax1.axvline(x=tc, color=cmap2(0.60), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.8)
# 	for tc in time_sig2_anti:
# 		ax1.axvline(x=tc, color=cmap2(0.60), alpha=alpha_fill, linewidth=LWIDTH, zorder=0.8)
	
	data_cluster = cc_err_all[ii]
	time = data_cluster[0]
	for jj in range(0,len(data_cluster[1])):
		cor_mean = data_cluster[1][jj]
		cor_err_d = data_cluster[2][jj]
		cor_err_u = data_cluster[3][jj]
		
		ax1.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_BK, facecolor='grey', edgecolor='grey', linewidth=ELINE, zorder=0.9)
# 		ax3.fill_between(time, cor_mean-cor_err_d, cor_mean+cor_err_u, alpha=ALPHA_BK, facecolor='grey', edgecolor='grey', linewidth=ELINE, zorder=0.9)
	

	ax0.set_xlim([-1299.999,2999.999])
	ax0.set_ylim([-0.399,0.999])

	ax1.set_xlim([-1299.999,2999.999])
	ax1.set_ylim([-0.399,0.499])

# 	ax2.set_xlim([-49.99,49.99])
# 	ax2.set_ylim([0.001,0.999])

# 	ax3.set_xlim([-49.99,49.99])
# 	ax3.set_ylim([0.001,0.399])

	ax0.text(-1000., 0.85, r'AC', verticalalignment='center', horizontalalignment='left', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))		
	ax0.text(150., 0.85, r'gamma-ray$\rightarrow$', verticalalignment='center', horizontalalignment='left', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))
	ax0.text(-150., 0.85, r'$\leftarrow$optical', verticalalignment='center', horizontalalignment='right', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))
	
	ax0.text(0.66, 0.66, 'Cluster '+LABEL, verticalalignment='center', horizontalalignment='center', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2), transform=ax0.transAxes, size=10, color=COLOR[0])	
	
	ax1.text(-1000., 0.4, r'CC', verticalalignment='center', horizontalalignment='left', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))		
# 	ax1.text(150., 0.4, r'gamma-ray leading$\rightarrow$', verticalalignment='center', horizontalalignment='left', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))
# 	ax1.text(-150., 0.4, r'$\leftarrow$optical leading', verticalalignment='center', horizontalalignment='right', bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=2))

	ax0.set_ylabel(r'$r_s$')
	ax1.set_ylabel(r'$r_s$')
# 	ax2.set_ylabel(r'$r_s$')
	ax1.set_xlabel(r'$\tau_\mathrm{offset}$ [days]')

	# ax1.yaxis.set_ticks_position('right')
# 	plt.setp(ax1.get_xticklabels(), ha='right', rotation=45)
	ax1.zorder=1000
	
# 	ax0.legend(loc='upper right')
	
	plt.tight_layout()
	out_folder = './individual_cluster_plots/'
	plot_name = out_folder+'plot_stacked_CC_cluster'+LABEL+'_'
	for n in range(0,100):
		if os.path.isfile(plot_name+str(n).zfill(2)+'.png'):
			continue
		else:
			plt.savefig(plot_name+str(n).zfill(2)+'.png', bbox_inches='tight', dpi=400)
			break
	plt.close()

